In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Dict, Literal, Union
import random
from itertools import cycle, islice
import traceback

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()
ngrok.set_auth_token("2wcLhKAkhn0MwFEwOlt2CXaabqk_4n42xCLtLqGuFKiKLfrik")

app = FastAPI(title="Colab Weekly Workout API")

In [ ]:
# Pydantic 모델 정의
class ExerciseItem(BaseModel):
    name: str
    default_equipment: str
    possible_equipment: List[str]
    sets: str

class BeginnerWorkoutRequest(BaseModel):
    email: str
    goal: Literal["muscle_gain", "fat_loss", "maintenance"]
    preferred_parts: List[str]
    gender: Literal["male", "female"]
    height: float
    weight: float
    level: Literal["beginner", "intermediate", "advanced"]
    frequency_per_week: int

class FeedbackItem(BaseModel):
    email: str
    exercise_name: str
    rating: Literal["good", "bad", "skip"]

In [ ]:
############# 피드백을 반영한 운동 루틴 추천 ###################
feedback_db: Dict[str, Dict[str, int]] = {}

@app.post("/feedback")
def receive_feedback(feedback: FeedbackItem):
    user_data = feedback_db.setdefault(feedback.email, {})
    score = {"good": +1, "bad": -1, "skip": -2}[feedback.rating]
    user_data[feedback.exercise_name] = user_data.get(feedback.exercise_name, 0) + score
    return {"message": "Feedback received."}

In [ ]:
# 목표별 세트x반복 매핑
rep_ranges = {
    "muscle_gain": "4x8",
    "fat_loss": "3x15",
    "maintenance": "3x12"
}

# rule-based 운동 DB(6개 부위, 각 5개 운동)
exercise_db: Dict[str, List[Dict[str, Union[str, List[str]]]]] = {
    "가슴": [
        {"name": "인클라인 벤치 프레스", "possible_equipment": ["바벨", "덤벨"]},
        {"name": "벤치 프레스 (와이드 그립)", "possible_equipment": ["바벨"]},
        {"name": "푸쉬업", "possible_equipment": ["맨몸", "덤벨"]},
        {"name": "덤벨 플라이", "possible_equipment": ["덤벨"]},
        {"name": "케이블 크로스오버", "possible_equipment": ["머신"]},
        {"name": "딥스", "possible_equipment": ["맨몸"]},
        {"name": "펙덱 플라이", "possible_equipment": ["머신"]},
        {"name": "클로즈 그립 벤치 프레스", "possible_equipment": ["바벨"]},
    ],
    "등": [
        {"name": "렛풀다운", "possible_equipment": ["머신"]},
        {"name": "바벨 로우", "possible_equipment": ["바벨"]},
        {"name": "풀업", "possible_equipment": ["맨몸"]},
        {"name": "시티드 케이블 로우", "possible_equipment": ["머신"]},
        {"name": "덤벨 풀오버", "possible_equipment": ["덤벨"]},
        {"name": "T바 로우", "possible_equipment": ["바벨"]},
        {"name": "원암 덤벨 로우", "possible_equipment": ["덤벨"]},
        {"name": "데드리프트", "possible_equipment": ["바벨"]},
    ],
    "하체": [
        {"name": "바벨 스쿼트", "possible_equipment": ["바벨"]},
        {"name": "레그 프레스", "possible_equipment": ["머신"]},
        {"name": "런지", "possible_equipment": ["덤벨", "맨몸"]},
        {"name": "레그 컬", "possible_equipment": ["머신"]},
        {"name": "덤벨 데드리프트", "possible_equipment": ["덤벨"]},
        {"name": "레그 익스텐션", "possible_equipment": ["머신"]},
        {"name": "스모 데드리프트", "possible_equipment": ["바벨"]},
        {"name": "불가리안 스플릿 스쿼트", "possible_equipment": ["덤벨", "맨몸"]},
    ],
    "전신": [
        {"name": "파머스 캐리", "possible_equipment": ["덤벨"]},
        {"name": "버피", "possible_equipment": ["맨몸"]},
        {"name": "스쿼트 투 프레스", "possible_equipment": ["덤벨"]},
        {"name": "케틀벨 스윙", "possible_equipment": ["덤벨"]},
        {"name": "마운틴 클라이머", "possible_equipment": ["맨몸"]},
        {"name": "메디신볼 슬램", "possible_equipment": ["머신"]},
        {"name": "배틀로프", "possible_equipment": ["기타"]},
        {"name": "맨몸 서킷 트레이닝", "possible_equipment": ["맨몸"]},
    ],
    "어깨": [
        {"name": "밀리터리 프레스", "possible_equipment": ["바벨", "덤벨"]},
        {"name": "사이드 레터럴 레이즈", "possible_equipment": ["덤벨"]},
        {"name": "오버헤드 프레스", "possible_equipment": ["머신"]},
        {"name": "프론트 레이즈", "possible_equipment": ["덤벨"]},
        {"name": "리버스 플라이", "possible_equipment": ["머신"]},
        {"name": "덤벨 숄더 프레스", "possible_equipment": ["덤벨"]},
        {"name": "페이스 풀", "possible_equipment": ["머신"]},
        {"name": "핸드스탠드 푸쉬업", "possible_equipment": ["맨몸"]},
    ],
    "팔": [
        {"name": "바벨 컬", "possible_equipment": ["바벨"]},
        {"name": "덤벨 컬", "possible_equipment": ["덤벨"]},
        {"name": "트라이셉스 푸시다운", "possible_equipment": ["머신"]},
        {"name": "덤벨 킥백", "possible_equipment": ["덤벨"]},
        {"name": "해머 컬", "possible_equipment": ["덤벨"]},
        {"name": "컨센트레이션 컬", "possible_equipment": ["덤벨"]},
        {"name": "스컬 크러셔", "possible_equipment": ["바벨"]},
        {"name": "케이블 오버헤드 익스텐션", "possible_equipment": ["머신"]},
    ],
    "복부": [
        {"name": "크런치", "possible_equipment": ["맨몸"]},
        {"name": "레그 레이즈", "possible_equipment": ["맨몸"]},
        {"name": "러시안 트위스트", "possible_equipment": ["맨몸", "덤벨"]},
        {"name": "플랭크", "possible_equipment": ["맨몸"]},
        {"name": "케이블 크런치", "possible_equipment": ["머신"]},
        {"name": "바이시클 크런치", "possible_equipment": ["맨몸"]},
        {"name": "V업", "possible_equipment": ["맨몸"]},
        {"name": "행잉 레그 레이즈", "possible_equipment": ["맨몸"]},
    ],
}

In [ ]:
# 초보자 필터링
def filter_by_level(candidates, level):
    if level == "beginner":
        return [ex for ex in candidates if any(eq in ["맨몸", "머신"] for eq in ex["possible_equipment"])]
    return candidates

def get_filtered_candidates(email, candidates):
    user_data = feedback_db.get(email, {})
    return [ex for ex in candidates if user_data.get(ex["name"], 0) > -2]

def weighted_sample(email, ex_list):
    user_data = feedback_db.get(email, {})
    weights = [max(1 + user_data.get(ex["name"], 0), 1) for ex in ex_list]
    return random.choices(ex_list, weights=weights, k=1)[0]

def generate_daily_routine(email, goal, level, part_list):
    routine = []
    added_names = set()

    # 1) 부위별 1개 추천
    for part in part_list:
        candidates = get_filtered_candidates(email, filter_by_level(exercise_db.get(part, []), level))
        if candidates:
            ex = weighted_sample(email, candidates)
            routine.append(ExerciseItem(
                name=ex["name"],
                default_equipment=ex["possible_equipment"][0],
                possible_equipment=ex["possible_equipment"],
                sets=rep_ranges[goal]
            ))
            added_names.add(ex["name"])

    # 2) 부족하면 해당 부위에서 보충
    pool = [
        ex for part in part_list
        for ex in get_filtered_candidates(email, filter_by_level(exercise_db.get(part, []), level))
        if ex["name"] not in added_names
    ]
    while len(routine) < 5 and pool:
        ex = weighted_sample(email, pool)
        if ex["name"] not in added_names:
            routine.append(ExerciseItem(
                name=ex["name"],
                default_equipment=ex["possible_equipment"][0],
                possible_equipment=ex["possible_equipment"],
                sets=rep_ranges[goal]
            ))
            added_names.add(ex["name"])
            pool = [e for e in pool if e["name"] not in added_names]

    # 3) 그래도 부족하면 전체 DB에서 보충
    if len(routine) < 5:
        all_exs = [
            ex for part in exercise_db.values()
            for ex in get_filtered_candidates(email, filter_by_level(part, level))
            if ex["name"] not in added_names
        ]
        while len(routine) < 5 and all_exs:
            ex = weighted_sample(email, all_exs)
            if ex["name"] not in added_names:
                routine.append(ExerciseItem(
                    name=ex["name"],
                    default_equipment=ex["possible_equipment"][0],
                    possible_equipment=ex["possible_equipment"],
                    sets=rep_ranges[goal]
                ))
                added_names.add(ex["name"])
                all_exs = [e for e in all_exs if e["name"] not in added_names]

    return routine

In [ ]:
@app.post(
    "/generate",
    response_model=Dict[str, Dict[str, Union[List[str], List[ExerciseItem]]]],
    summary="요일별 운동 루틴 추천"
)
def generate(req: BeginnerWorkoutRequest):
    parts = req.preferred_parts.copy()
    days = req.frequency_per_week
    if days <= 0 or not parts:
        raise HTTPException(400, "부위와 주당 운동 횟수는 1개 이상이어야 합니다.")

    # 1) 부위 분배: preferred_parts vs days 관계에 따라 자동 분기
    large_muscles = {"가슴", "등", "하체", "어깨"}

    if len(parts) <= days:
        # 부위 개수 ≤ 일수: 하루 1개씩 순환 할당
        bins = [[part] for part in islice(cycle(parts), days)]
    else:
        # 부위 개수 > 일수: divmod → 최대 2개, 대/소근육 조합
        base, rem = divmod(len(parts), days)
        sizes = [base + (1 if i < rem else 0) for i in range(days)]
        large = [p for p in parts if p in large_muscles]
        small = [p for p in parts if p not in large_muscles]
        bins = []
        for sz in sizes:
            grp = []
            if sz == 2 and small and large:
                grp += [small.pop(0), large.pop(0)]
            else:
                for _ in range(sz):
                    if large: grp.append(large.pop(0))
                    elif small: grp.append(small.pop(0))
            bins.append(grp)

    # 2) 같은 대근육 연속 방지
    for _ in range(50):
        if all(not(set(bins[i]) & set(bins[i+1]) & large_muscles) for i in range(len(bins)-1)):
            break
        random.shuffle(bins)

    # 3) 최종 스케줄 생성
    schedule = {
        f"Day{i+1}": {
            "target_parts": group,
            "exercises": generate_daily_routine(req.email, req.goal, req.level, group)
        } for i, group in enumerate(bins)
    }

    return schedule

In [ ]:
public_url = ngrok.connect(8000)
print(f"Swagger UI: {public_url}/docs")

uvicorn.run(app, host="0.0.0.0", port=8000)

Swagger UI: NgrokTunnel: "https://30b7-34-85-141-97.ngrok-free.app" -> "http://localhost:8000"/docs


INFO:     Started server process [2001]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     1.241.21.88:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "POST /feedback HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "POST /feedback HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "POST /generate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2001]
